In [1]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from sklearn.cluster import KMeans

In [23]:
df = pd.read_csv("data3888/data/data.txt", index_col = 0)
df_new = df.drop(columns=['new_cases_smoothed_per_million', "human_development_index"])
df_new["date"] = pd.to_datetime(df_new["date"])
feb = (df_new['date'] >= '2022-02-01') & (df_new['date'] < '2022-03-01')
df_feb = df_new.loc[feb]
feb_grouped = df_feb.groupby("location").agg({'new_cases_per_million':'mean','stringency_index':'mean','positive_rate':'mean',"cost_living_index":"mean", "tourist_service_index":"mean", "international_travel_controls":"mean"})

scaled_df = pd.read_csv("data3888/scaled_with_clusters.csv", index_col = 0)

In [3]:
style = {'description_width': 'initial'}
layout = {"width" : "99%"}

In [4]:
#country dropdown
countries = list(df["location"].unique())
country_dropdown = widgets.Dropdown(
    options = countries,
    value = "Australia",
    description = "Starting country",
    style = style,
    layout = layout)

In [5]:
#continent dropdown
continents = list(df["continent"].unique())
continents.insert(0, "All")
continent_dropdown = widgets.Dropdown(options = continents,
                                      value = "All",
                                      description = "Continent",
                                      style = style,
                                      layout = layout)

In [6]:
#cases upper limit intslider
newcase_intslider = widgets.IntSlider(value = 100,
                                      min = 0,
                                      max = 1000,
                                      step = 50,
                                      description = "Maximum cases per million",
                                      style = style,
                                      layout = layout)

In [7]:
#stringency index intslider
stringency_intslider = widgets.IntSlider(value = 100,
                                         min = 0,
                                         max = 100,
                                         step = 50,
                                         description = "Maximum stringency index",
                                         style = style, 
                                         layout = layout)

In [8]:
#max positive rate floatslider
positivityrate_floatslider = widgets.FloatSlider(value = 100,
                                                 min = 0,
                                                 max = 1,
                                                 step = 0.01,
                                                 description = "Maximum positivity rate",
                                                 readout_format='.0%',
                                                 style = style,
                                                 layout = layout)

In [9]:
#COL intslider
costliving_intrangeslider = widgets.IntRangeSlider(value = [0, 100],
                                                   min = 0,
                                                   max = 100,
                                                   description = "Cost of living",
                                                   style = style,
                                                   layout = layout)

In [10]:
#tourist service floatrangeslider
touristservice_floatrangeslider = widgets.FloatRangeSlider(value = [1,7],
                                                           min = 1,
                                                           max = 7,
                                                           step = 0.25,
                                                           description = "Tourist Service Index",
                                                           style = style,
                                                           layout = layout)

In [11]:
#international travel controls intslider
travelcontrols_intslider = widgets.IntSlider(value = 0,
                                             min = 0,
                                             max = 4,
                                             description = "Max Travel Controls",
                                             style = style,
                                             layout = layout)

In [12]:
#covid box
covid_box = widgets.VBox([newcase_intslider, positivityrate_floatslider, stringency_intslider])

In [13]:
#travel box
travel_box = widgets.VBox([costliving_intrangeslider, touristservice_floatrangeslider, travelcontrols_intslider, continent_dropdown])

In [14]:
#Simple settings box
run_btn = widgets.Button(description = "Run",
                         layout = layout)
simple_box = widgets.HBox([country_dropdown, run_btn])

In [15]:
#advanced settings tab
advanced_tab = widgets.Tab([covid_box, travel_box])
advanced_tab.set_title(0, "Covid Settings")
advanced_tab.set_title(1, "Travel Settings")

In [16]:
#advanced settings accordion
advanced_accordion = widgets.Accordion([advanced_tab])
advanced_accordion.set_title(0, "Advanced Settings")

In [17]:
#final settings ui
settings_ui = widgets.VBox([simple_box, advanced_accordion])
settings_ui

In [18]:
#define interactivity
def run(x):
    result_box = setup_dfbox()
    ymal_box = setup_ymalbox()
    selected_country_heading = widgets.HTML('<h2 style="color:black" > {}\'s Stats'.format(country_dropdown.value))
    ymal_heading = widgets.HTML('<h2 style="color:black" > You may also like')
    
    settings_ui.children = [simple_box, advanced_accordion, selected_country_heading, result_box, ymal_heading, ymal_box]
    
def setup_dfbox():
    selected_country = feb_grouped.loc[[country_dropdown.value]]
    out = widgets.Output()
    with out:
        display(selected_country)
    return out

def setup_ymalbox():
    cluster = scaled_df[scaled_df["location"] == country_dropdown.value].iloc[0,3]
    ymal_df = scaled_df[scaled_df["hcluster"] == cluster]
    out = widgets.Output()
    with out:
        display(ymal_df)
    return out

run_btn.on_click(run)

In [19]:
feb_grouped

,new_cases_per_million,stringency_index,positive_rate,cost_living_index,tourist_service_index,international_travel_controls
location,,,,,,
Albania,4531.952,61.110000,0.519526,35.50,4.01,3.0
Algeria,287.317,59.653214,NaN,26.87,1.84,3.0
Argentina,11445.908,35.190000,0.229107,34.69,4.53,2.0
Armenia,17531.926,NaN,0.300214,33.89,4.33,NaN
Australia,25484.702,53.177143,0.271000,77.75,6.06,3.0
...,...,...,...,...,...,...
Venezuela,1015.502,28.700000,NaN,43.20,2.96,3.0
Vietnam,11895.407,65.280000,0.500989,37.48,2.85,2.0
Yemen,24.664,12.040000,NaN,53.14,1.87,1.0


In [20]:
scaled_df

,iso_code,new_cases_smoothed_per_million,tourist_service_index,hcluster,location
0,ALB,86.7700,4.01,1,Albania
1,ARE,98.9455,5.63,4,United Arab Emirates
2,ARG,151.1790,4.53,5,Argentina
3,ARM,119.0265,4.33,5,Armenia
4,AUS,1.0750,6.06,2,Australia
...,...,...,...,...,...
111,VNM,0.1130,2.85,1,Vietnam
112,YEM,0.2340,1.87,1,Yemen
113,ZAF,41.2240,4.30,2,South Africa
114,ZMB,6.6290,2.52,1,Zambia


In [21]:
country_dropdown.value

'Australia'

In [22]:
scaled_df[scaled_df["hcluster"] == scaled_df[scaled_df["location"] == country_dropdown.value].iloc[0,3]]

,iso_code,new_cases_smoothed_per_million,tourist_service_index,hcluster,location
4,AUS,1.0750,6.06,2,Australia
13,BRB,25.3230,4.47,2,Barbados
25,DOM,50.7460,4.82,2,Dominican Republic
32,FIN,46.0370,4.66,2,Finland
39,HKG,1.0590,4.27,2,Hong Kong
47,ISL,35.8315,6.00,2,Iceland
50,JAM,26.2320,4.90,2,Jamaica
52,JPN,9.7535,5.71,2,Japan
56,KOR,10.0605,5.55,2,South Korea
65,MEX,40.6530,4.81,2,Mexico
